In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [38]:
appl = yf.download('AAPL', start='2019-01-01', end='2025-03-05')

[*********************100%***********************]  1 of 1 completed


In [39]:
appl.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in appl.columns
]

In [40]:
def calculate_vortex(df, value, n):
    """Calculate Vortex Indicator VI+ and VI-."""
    high = df[("High_"+value)]
    low = df[("Low_"+value)]
    close = df[("Close_"+value)]

    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [41]:
appl['VI+_'], appl['VI-_'] = calculate_vortex(appl, 'AAPL', 14)

In [42]:
import json
import pandas as pd

# Load from file
with open("AAPL_sentiment_raw.json", "r") as f:
    sentiment_json = json.load(f)

# Extract feed
sentiment_feed = sentiment_json.get("feed", [])

# Extract useful fields
sentiment_data = []

for item in sentiment_feed:
    try:
        sentiment_data.append({
            "time_published": pd.to_datetime(item["time_published"]),
            "sentiment_score": float(item["overall_sentiment_score"]),
            "sentiment_label": item["overall_sentiment_label"],
        })
    except (KeyError, ValueError, TypeError):
        continue  # Skip malformed rows

# Convert to DataFrame
sentiment_df = pd.DataFrame(sentiment_data)
sentiment_df.set_index("time_published", inplace=True)

# View result
print(sentiment_df.head())
print(sentiment_df.info())

                     sentiment_score   sentiment_label
time_published                                        
2025-03-01 00:00:18         0.225994  Somewhat-Bullish
2025-02-28 22:06:00         0.291136  Somewhat-Bullish
2025-02-28 17:55:55         0.082801           Neutral
2025-02-28 17:00:45         0.374552           Bullish
2025-02-28 15:00:46         0.287114  Somewhat-Bullish
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 669 entries, 2025-03-01 00:00:18 to 2025-01-15 14:45:51
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sentiment_score  669 non-null    float64
 1   sentiment_label  669 non-null    object 
dtypes: float64(1), object(1)
memory usage: 15.7+ KB
None


In [43]:
sentiment_data = []
for news_item in sentiment_json.get("feed", []):
    sentiment_data.append({
            "time_published": pd.to_datetime(news_item["time_published"]),
            "sentiment_score": news_item["overall_sentiment_score"],
            "sentiment_label": news_item["overall_sentiment_label"],
    })
sentiment_data = pd.DataFrame(sentiment_data)
sentiment_data['time_published'] = sentiment_data['time_published'].dt.date


In [44]:
sentiment_scores_filtered = sentiment_data[pd.to_datetime(sentiment_data['time_published']).isin(appl.index)]
sentiment_scores_filtered = sentiment_scores_filtered.groupby('time_published')['sentiment_score'].mean().reset_index()
print(len(sentiment_scores_filtered))

31


In [45]:
appl_volume = appl[('Volume_AAPL')].reset_index()
appl_volume['Date'] = pd.to_datetime(appl_volume['Date'])

sentiment_scores_filtered['time_published'] = pd.to_datetime(sentiment_scores_filtered['time_published'])

merged_data = pd.merge(appl_volume, sentiment_scores_filtered, left_on='Date', right_on='time_published', how='inner')
merged_data['Weighted_Sentiment'] = merged_data['Volume_AAPL'] * merged_data['sentiment_score']
merged_data['5_day_avg_sentiment'] = merged_data['Weighted_Sentiment'].rolling(window=5).mean()
merged_data['Buy_Condition'] = merged_data['5_day_avg_sentiment'] > 0
merged_data['5_day_avg_sentiment_norm'] = merged_data['5_day_avg_sentiment']/merged_data['Volume_AAPL'].mean()

merged_data.head()

,Date,Volume_AAPL,time_published,sentiment_score,Weighted_Sentiment,5_day_avg_sentiment,Buy_Condition,5_day_avg_sentiment_norm
0,2025-01-15,39832000,2025-01-15,0.223177,8.889575e+06,NaN,False,NaN
1,2025-01-16,71759100,2025-01-16,0.237567,1.704756e+07,NaN,False,NaN
2,2025-01-17,68488300,2025-01-17,0.130304,8.924326e+06,NaN,False,NaN
3,2025-01-21,98070400,2025-01-21,0.169273,1.660064e+07,NaN,False,NaN
4,2025-01-22,64126500,2025-01-22,0.182421,1.169803e+07,1.263203e+07,True,0.231401


## volatility

In [46]:
# Calculate True Range
appl["prev_close"] = appl["Close_AAPL"].shift(1)
appl["tr1"] = appl["High_AAPL"] - appl["Low_AAPL"]
appl["tr2"] = abs(appl["High_AAPL"] - appl["prev_close"])
appl["tr3"] = abs(appl["Low_AAPL"] - appl["prev_close"])

appl["true_range"] = appl[["tr1", "tr2", "tr3"]].max(axis=1)

# 10-day ATR
appl["ATR_10"] = appl["true_range"].rolling(window=10).mean()

# ---- STEP 4: Calculate ATR as a percentage of closing price ----
appl["atr_pct"] = appl["ATR_10"] / appl["Close_AAPL"]

# allocating the capital

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

appl["position_size"] = appl.apply(position_size, axis=1)

print(appl[["Close_AAPL", "ATR_10", "atr_pct", "position_size"]].tail(10))


            Close_AAPL    ATR_10   atr_pct  position_size
Date                                                     
2025-02-19  244.869995  4.939392  0.020171           0.01
2025-02-20  245.830002  4.735891  0.019265           0.01
2025-02-21  245.550003  4.746260  0.019329           0.01
2025-02-24  247.100006  4.517000  0.018280           0.01
2025-02-25  247.039993  4.687000  0.018973           0.01
2025-02-26  240.360001  4.719998  0.019637           0.01
2025-02-27  237.300003  4.631998  0.019520           0.01
2025-02-28  241.839996  5.143999  0.021270           0.01
2025-03-03  238.029999  5.479999  0.023022           0.01
2025-03-04  235.929993  5.685001  0.024096           0.01


In [47]:
import plotly.express as px
fig = px.line(appl, x=appl.index, y="atr_pct", title="AAPL ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()

In [48]:
# Filter only 2025 data
appl_2025 = appl[appl.index.year == 2025]
fig = px.line(appl_2025, x=appl_2025.index, y="atr_pct", title="AAPL ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()

In [49]:
merged_data = pd.merge(merged_data, appl, on='Date', how='left')

In [50]:
# Calculate ATR percentage
merged_data['atr_pct'] = merged_data['ATR_10'] / merged_data['Close_AAPL']

# Vortex crossover logic
merged_data['VI_Cross_Up'] = (merged_data['VI+_'] > merged_data['VI-_']) & (merged_data['VI+_'].shift(1) <= merged_data['VI-_'].shift(1))
merged_data['VI_Cross_Down'] = (merged_data['VI-_'] > merged_data['VI+_']) & (merged_data['VI-_'].shift(1) <= merged_data['VI+_'].shift(1))

# Initialize signal & state columns
merged_data['Buy_Signal'] = False
merged_data['Sell_Signal'] = False
merged_data['Position'] = 0
merged_data['Entry_Type'] = None  # aggressive/conservative

# Trailing stop logic variables
in_position = False
peak_price = 0

for i in range(1, len(merged_data)):
    row = merged_data.iloc[i]
    idx = merged_data.index[i]
    # Buy condition
    if not in_position or row['VI_Cross_Up'] or row['5_day_avg_sentiment_norm']>0:
        merged_data.at[idx, 'Buy_Signal'] = True
        merged_data.at[idx, 'Position'] = 1
        in_position = True
        peak_price = row['Close_AAPL']

        # Entry Type: aggressive if ATR < 3%, else conservative
        if row['atr_pct'] < 0.03:
            merged_data.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            merged_data.at[idx, 'Entry_Type'] = 'conservative'

    # While in position, check for trailing stop or VI cross down
    elif in_position:
        current_price = row['Close_AAPL']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03 or row['VI_Cross_Down']:
            merged_data.at[idx, 'Sell_Signal'] = True
            merged_data.at[idx, 'Position'] = 0
            in_position = False
        else:
            merged_data.at[idx, 'Position'] = 1

# Show result counts
print("Buy signals:", merged_data['Buy_Signal'].sum())
print("Sell signals:", merged_data['Sell_Signal'].sum())
print("Aggressive entries:", (merged_data['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (merged_data['Entry_Type'] == 'conservative').sum())

Buy signals: 28
Sell signals: 1
Aggressive entries: 23
Conservative entries: 5


In [51]:
import plotly.graph_objects as go

fig = go.Figure()

# Plot merged_data closing price
fig.add_trace(go.Scatter(
    x=merged_data.index, 
    y=merged_data['Close_AAPL'], 
    mode='lines', 
    name='merged_data Price', 
    line=dict(color='blue')
))

# Aggressive buys
fig.add_trace(go.Scatter(
    x=merged_data[(merged_data['Buy_Signal']) & (merged_data['Entry_Type'] == 'aggressive')].index,
    y=merged_data[(merged_data['Buy_Signal']) & (merged_data['Entry_Type'] == 'aggressive')]['Close_AAPL'],
    mode='markers',
    name='Buy (Aggressive)',
    marker=dict(symbol='triangle-up', color='limegreen', size=10)
))

# Conservative buys
fig.add_trace(go.Scatter(
    x=merged_data[(merged_data['Buy_Signal']) & (merged_data['Entry_Type'] == 'conservative')].index,
    y=merged_data[(merged_data['Buy_Signal']) & (merged_data['Entry_Type'] == 'conservative')]['Close_AAPL'],
    mode='markers',
    name='Buy (Conservative)',
    marker=dict(symbol='triangle-up', color='green', size=10)
))

# Sells
fig.add_trace(go.Scatter(
    x=merged_data[merged_data['Sell_Signal']].index,
    y=merged_data[merged_data['Sell_Signal']]['Close_AAPL'],
    mode='markers',
    name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=10)
))

fig.update_layout(
    title='merged_data Buy/Sell Signals Over Time',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    height=600
)

fig.show()

In [52]:
import plotly.graph_objects as go

# Ensure 'Date' is datetime and set as index if needed
merged_data['Date'] = pd.to_datetime(merged_data['Date'])

fig = go.Figure()

# Plot 5-day Avg Sentiment
fig.add_trace(go.Scatter(
    x=merged_data['Date'],
    y=merged_data['5_day_avg_sentiment_norm'],
    mode='lines+markers',
    name='5-Day Avg Sentiment',
    line=dict(color='blue')
))

# Plot ATR %
fig.add_trace(go.Scatter(
    x=merged_data['Date'],
    y=merged_data['atr_pct'],
    mode='lines+markers',
    name='ATR %',
    yaxis='y2',
    line=dict(color='orange')
))

# Optional: Highlight Buy Signal Dates (even though there are none now)
fig.add_trace(go.Scatter(
    x=merged_data.loc[merged_data['Buy_Signal'], 'Date'],
    y=merged_data.loc[merged_data['Buy_Signal'], '5_day_avg_sentiment_norm'],
    mode='markers',
    marker=dict(color='green', size=10, symbol='star'),
    name='Buy Signal'
))

# Add dual axis layout
fig.update_layout(
    title="5-Day Sentiment vs ATR % (with Buy Signals)",
    xaxis_title='Date',
    yaxis=dict(title='5-Day Avg Sentiment'),
    yaxis2=dict(title='ATR %', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    height=500
)

fig.show()

In [53]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=merged_data.index, y=merged_data['Close_AAPL'], mode='lines', name='merged_data Price'))

# Buy markers
fig.add_trace(go.Scatter(
    x=merged_data[merged_data['Buy_Signal']].index,
    y=merged_data[merged_data['Buy_Signal']]['Close_AAPL'],
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    name='Buy Signal'
))

# Sell markers
fig.add_trace(go.Scatter(
    x=merged_data[merged_data['Sell_Signal']].index,
    y=merged_data[merged_data['Sell_Signal']]['Close_AAPL'],
    mode='markers',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    name='Sell Signal'
))

fig.update_layout(title='XLY Buy & Sell Signals', template='plotly_white')
fig.show()

In [54]:
capital = 100000
in_position = False
entry_price = 0
position_value = 0
cash = capital
returns = []

for i in range(len(merged_data)):
    row = merged_data.iloc[i]
    
    # Buy
    if row['Buy_Signal'] and not in_position:
        position_size = row['position_size']
        position_value = cash * position_size
        entry_price = row['Close_AAPL']
        shares_bought = position_value / entry_price
        cash -= position_value
        in_position = True
        
    # Sell
    elif row['Sell_Signal'] and in_position:
        exit_price = row['Close_AAPL']
        proceeds = shares_bought * exit_price
        profit = proceeds - position_value
        cash += proceeds
        returns.append(profit)
        in_position = False
        position_value = 0
        entry_price = 0

# Final capital
final_value = cash + (shares_bought * row['Close_AAPL'] if in_position else 0)
total_return = final_value - capital

print(f"Final Capital: ${final_value:.2f}")
print(f"Total Return: ${total_return:.2f}")
print(f"Total Trades: {len(returns)}")
print(f"Average Profit per Trade: ${np.mean(returns):.2f}")

Final Capital: $100057.01
Total Return: $57.01
Total Trades: 1
Average Profit per Trade: $-24.62


In [59]:
import vectorbt as vbt

# Make sure index is datetime and 'Close_TSLA' exists
price = merged_data['Close_AAPL']

# Generate entries and exits from your signals
entries = merged_data['Buy_Signal']
exits = merged_data['Sell_Signal']

# Create portfolio
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    size=np.nan,  # Let it auto-calculate position size if fixed capital
    init_cash=100_000,
    fees=0.001,  # 0.1% per trade
    slippage=0.0005  # Optional
)

# Plot portfolio value
print(portfolio.stats())
portfolio.plot().show()

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                              0.000000
End                               30.000000
Period                            31.000000
Start Value                   100000.000000
End Value                     100000.000000
Total Return [%]                   0.000000
Benchmark Return [%]               1.780762
Max Gross Exposure [%]             0.000000
Total Fees Paid                    0.000000
Max Drawdown [%]                        NaN
Max Drawdown Duration                   NaN
Total Trades                       0.000000
Total Closed Trades                0.000000
Total Open Trades                  0.000000
Open Trade PnL                     0.000000
Win Rate [%]                            NaN
Best Trade [%]                          NaN
Worst Trade [%]                         NaN
Avg Winning Trade [%]                   NaN
Avg Losing Trade [%]                    NaN
Avg Winning Trade Duration              NaN
Avg Losing Trade Duration               NaN
Profit Factor                   

In [60]:
appl_ = merged_data.dropna(subset=['Close_AAPL'])
entries = merged_data['Buy_Signal'].astype(bool)
exits = merged_data['Sell_Signal'].astype(bool)

price = merged_data['Close_AAPL']
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

print(portfolio.stats())
portfolio.plot().show()



Start                              0.000000
End                               30.000000
Period                            31.000000
Start Value                   100000.000000
End Value                     105185.963339
Total Return [%]                   5.185963
Benchmark Return [%]               1.780762
Max Gross Exposure [%]           100.000000
Total Fees Paid                  294.586321
Max Drawdown [%]                   4.900568
Max Drawdown Duration             10.000000
Total Trades                       2.000000
Total Closed Trades                1.000000
Total Open Trades                  1.000000
Open Trade PnL                  7842.950122
Win Rate [%]                       0.000000
Best Trade [%]                    -2.659644
Worst Trade [%]                   -2.659644
Avg Winning Trade [%]                   NaN
Avg Losing Trade [%]              -2.659644
Avg Winning Trade Duration              NaN
Avg Losing Trade Duration          2.000000
Profit Factor                   

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



# Without Sentiment

In [55]:
# WITHOUT sentiment
appl_copy = appl.copy()
appl_copy['atr_pct'] = appl_copy['ATR_10'] / appl_copy['Close_AAPL']

# Create Buy Signal (assuming VI_Cross_Up is defined elsewhere)
appl_copy['Buy_Signal'] = appl_copy['VI+_'] > appl_copy['VI-_']  # Vortex crossover
# + add any other buy conditions here...

# Create Sell Signal (basic)
appl_copy['Sell_Signal'] = appl_copy['VI-_'] > appl_copy['VI+_']

# Initialize position state
appl_copy['Position'] = 0
peak_price = 0

for i in range(1, len(appl_copy)):
    if appl_copy['Buy_Signal'].iloc[i]:
        appl_copy.at[appl_copy.index[i], 'Position'] = 1
        peak_price = appl_copy['Close_AAPL'].iloc[i]
    elif appl_copy['Position'].iloc[i - 1] == 1:
        current_price = appl_copy['Close_AAPL'].iloc[i]
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03:
            appl_copy.at[appl_copy.index[i], 'Sell_Signal'] = True  # trailing stop
            appl_copy.at[appl_copy.index[i], 'Position'] = 0
        else:
            appl_copy.at[appl_copy.index[i], 'Position'] = 1


In [56]:
capital = 100000
in_position = False
entry_price = 0
position_value = 0
cash = capital
returns = []

for i in range(len(appl_copy)):
    row = appl_copy.iloc[i]
    
    # Buy
    if row['Buy_Signal'] and not in_position:
        position_size = row['position_size']
        position_value = cash * position_size
        entry_price = row['Close_AAPL']
        shares_bought = position_value / entry_price
        cash -= position_value
        in_position = True
        
    # Sell
    elif row['Sell_Signal'] and in_position:
        exit_price = row['Close_AAPL']
        proceeds = shares_bought * exit_price
        profit = proceeds - position_value
        cash += proceeds
        returns.append(profit)
        in_position = False
        position_value = 0
        entry_price = 0

# Final capital
final_value = cash + (shares_bought * row['Close_AAPL'] if in_position else 0)
total_return = final_value - capital

print(f"Final Capital: ${final_value:.2f}")
print(f"Total Return: ${total_return:.2f}")
print(f"Total Trades: {len(returns)}")
print(f"Average Profit per Trade: ${np.mean(returns):.2f}")


Final Capital: $101622.75
Total Return: $1622.75
Total Trades: 65
Average Profit per Trade: $24.87


In [57]:
import vectorbt as vbt

appl = appl_copy.dropna(subset=['Close_AAPL'])
entries = appl_copy['Buy_Signal'].astype(bool)
exits = appl_copy['Sell_Signal'].astype(bool)

price = appl_copy['Close_AAPL']
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

print(portfolio.stats())
portfolio.plot().show()

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                              100000.0
End Value                           387826.049812
Total Return [%]                        287.82605
Benchmark Return [%]                   526.354164
Max Gross Exposure [%]                      100.0
Total Fees Paid                       35703.51859
Max Drawdown [%]                        20.871711
Max Drawdown Duration                       351.0
Total Trades                                   66
Total Closed Trades                            65
Total Open Trades                               1
Open Trade PnL                        4492.821209
Win Rate [%]                            44.615385
Best Trade [%]                           41.28428
Worst Trade [%]                        -11.056941
Avg Winning Trade [%]                    8.410415
Avg Losing Trade [%]                    -2.459743


In [33]:
# Calculate ATR percentage
appl['atr_pct'] = appl['ATR_10'] / appl['Close_AAPL']
appl['Buy_Signal'] = appl['VI+_'] > appl['VI-_']  # Vortex crossover
appl['Sell_Signal'] = appl['VI-_'] > appl['VI+_']

# Initialize position state
appl['Position'] = 0
appl['Entry_Type'] = None
peak_price = 0

for i in range(1, len(appl)):
    if appl['Buy_Signal'].iloc[i]:
        appl.at[appl.index[i], 'Position'] = 1
        peak_price = appl['Close_AAPL'].iloc[i]
    elif appl['Position'].iloc[i - 1] == 1:
        current_price = appl['Close_AAPL'].iloc[i]
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03:
            appl.at[appl.index[i], 'Sell_Signal'] = True  # trailing stop
            appl.at[appl.index[i], 'Position'] = 0
        else:
            appl.at[appl.index[i], 'Position'] = 1

print("Buy signals:", appl['Buy_Signal'].sum())
print("Sell signals:", appl['Sell_Signal'].sum())

Buy signals: 986
Sell signals: 551


In [34]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=appl.index, y=appl['Close_AAPL'], mode='lines', name='AAPL Price'))

# Buy markers
fig.add_trace(go.Scatter(
    x=appl[appl['Buy_Signal']].index,
    y=appl[appl['Buy_Signal']]['Close_AAPL'],
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    name='Buy Signal'
))

# Sell markers
fig.add_trace(go.Scatter(
    x=appl[appl['Sell_Signal']].index,
    y=appl[appl['Sell_Signal']]['Close_AAPL'],
    mode='markers',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    name='Sell Signal'
))

fig.update_layout(title='AAPL Buy & Sell Signals', template='plotly_white')
fig.show()